# Estimate sample phases using ML
- 

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import itk
import itkwidgets
from itkwidgets import view, compare

In [6]:
from glob import glob
slices = glob('../images/concrete/*.tiff')
slices.sort()
concrete = itk.imread(slices)
print(itk.size(concrete))
print(itk.spacing(concrete))

itkSize3 ([2560, 2560, 11])
itkVectorD3 ([1, 1, 1])


In [7]:
new_spacing = [1, 1, 2] #change slice thickness
concrete.SetSpacing(new_spacing)

In [8]:
myViewer = view(concrete, mode='z') #slicing over z
myViewer


Viewer(geometries=[], gradient_opacity=0.22, mode='z', point_sets=[], rendered_image=<itkImagePython.itkImageF…

In [19]:
#These are the coordinates of your crop
myViewer.roi_region()


itkImageRegion3([436, 469, 0], [1646, 1766, 11])

In [28]:
# User-selection is input for slicing np array
np_concrete = itk.array_from_image(concrete) 
roi = myViewer.roi_slice()
np_coreSample = np_concrete[roi]
view(np_coreSample, annotations=False, ui_collapsed=True)

Viewer(annotations=False, geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.…

In [30]:
coreSample = itk.GetImageFromArray(np_coreSample)
concrete = itk.median_image_filter(coreSample)

ValueError: Array memory is not contiguous. Try converting your array with `ascontiguousarray()` or `copy()` or use `GetImageFromArray()`

In [17]:
#Convert itk image to numpy
np_img = itk.GetArrayFromImage(img) 

In [41]:
arr = itk.array_from_image(img)
roi = myViewer.roi_slice()
view(arr[roi], cmap=itkwidgets.cm.PrGn, annotations=False, ui_collapsed=True)

Viewer(annotations=False, cmap='PRGn', geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<it…

## Thresholding: exploring threshold using mid slice

In [18]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

middle = 100
aslice = np_img[:,:,middle]
def myThresh(t=128):
    thresholded = aslice>t
    plt.imshow(thresholded, cmap='gray')
    
interact(myThresh, t=widgets.IntSlider(min=0,max=255,step=1,value=128));

interactive(children=(IntSlider(value=128, description='t', max=255), Output()), _dom_classes=('widget-interac…

In [21]:
help(itk.threshold_image_filter) #todo: no relevant info

Help on function threshold_image_filter in module itkThresholdImageFilterPython:

threshold_image_filter(*args, **kwargs)
    Set image values to a user-specified value if they are below, above,
    or between simple threshold values.
    
    ThresholdImageFilter sets image values to a user-specified "outside"
    value (by default, "black") if the image values are below, above, or
    between simple threshold values.
    
    The available methods are:
    
    ThresholdAbove(): The values greater than the threshold value are set
    to OutsideValue
    
    ThresholdBelow(): The values less than the threshold value are set to
    OutsideValue
    
    ThresholdOutside(): The values outside the threshold range (less than
    lower or greater than upper) are set to OutsideValue
    
    Note that these definitions indicate that pixels equal to the
    threshold value are not set to OutsideValue in any of these methods
    
    The pixels must support the operators >= and <=.
    
    

In [47]:
img_thresh = itk.threshold_image_filter(img,) #TODO: inside_value=0,outside_value=1 don't work here
compare(img, img_thresh, link_cmap=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [43]:
img_thresh = itk.iso_data_threshold_image_filter(img,) #TODO: why this operation made data 2^15 ?
view(img_thresh,cmap=itkwidgets.cm.BrBG)

Viewer(cmap='BrBG', geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkIma…

In [30]:
compare(img,img_thresh) #todo: only second panel works

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…